In [2]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [3]:
#CREATE CSV FILE FOR NIFTI
#PATIENT NAME  |  STUDY UID  | CT PATH  |  PET PATH  |  MASK PATH 

csv_pet0_path = '/media/salim/DD 2To/AHL2011_CSV_DATA/pet0.csv'

with open(csv_pet0_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [20]:
print(len(csv_data))
list_nifti = os.listdir('/media/salim/DD 2To/AHL2011_NIFTI')

print(len(list_nifti))

796
1738


In [25]:
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI' 
new_liste = []
for liste in list_nifti :
    if os.path.isfile(os.path.join(nifti_directory, liste)) == True and 'AHL' not in liste: 
        new_liste.append(liste)

print(len(new_liste))

1734


In [28]:
new_dataset = []
for liste in new_liste : 
    study_uid = liste.split('_')[0]

    for data in dataset : 
        if study_uid in data[0] :
            new_dataset.append(data)

print(len(new_dataset))

1734


In [30]:
complete_dataset = []
complete_dataset.append(new_dataset[0])
for i in range(1, len(new_dataset)) :
    if new_dataset[i] not in complete_dataset : 
        complete_dataset.append(new_dataset[i])

print(len(complete_dataset))

write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_final', complete_dataset)

578


In [7]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/json/AHL2011_list_path_csv.json'
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)


dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

In [ ]:
#CLEAN TO WRITE THE CSV 

In [11]:
all_liste = []
for i in range(len(csv_data)) : 
    subliste = []
    subliste.append(csv_data[i][0])
    study_uid = csv_data[i][2]
    subliste.append(study_uid)
    for nifti in list_nifti : 
        if study_uid in nifti : 
            subliste.append(nifti)

    all_liste.append(subliste)

In [12]:
#si pas de nifti, on enleve les patients
no_nifti  = []
for liste in all_liste : 
    if len(liste) != 5 : 
        no_nifti.append(liste)

In [13]:
print(len(no_nifti))

166


In [14]:
for serie in no_nifti : 
    all_liste.remove(serie)

In [15]:
print(len(all_liste))

630


In [16]:
#enlever doublon 
new_liste = []
new_liste.append(all_liste[0])
for i in range(len(all_liste)) : 
    if all_liste[i] != new_liste[-1] : 
        new_liste.append(all_liste[i])


In [17]:
print(len(new_liste))

578


In [18]:
#TRIER LA LISTE POUR ENSUITE ECRIRE LE CSV
sorted_serie = []
for serie in new_liste : 
    sorted_serie.append(sorted(serie))
print(len(sorted_serie))


578


In [19]:
#ECRIRE CSV 

nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'

with open(os.path.join(nifti_directory, 'AHL2011_PET0_NIFTI_VERSION_2.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT NAME", "STUDY UID", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK"])
    for serie in sorted_serie : 
        csv_writer.writerow([serie[4], serie[0], os.path.join(nifti_directory,serie[1]), os.path.join(nifti_directory,serie[2]), os.path.join(nifti_directory,serie[3])])
